# Image Feature Extraction

In [ ]:
import altair as alt
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

import sys

sys.path.append('../')

alt.renderers.enable('default')

In [ ]:
import json
import os

folder_path = './datasets/Caltech-101-ObjectCategories'

# Note: Caltech-101 dataset images are not of the same size

X_raw = []
for filename in os.listdir(f'{folder_path}/imgs'):
    X_raw.append(cv.imread(f'{folder_path}/imgs/{filename}'))
X_raw = np.array(X_raw)

with open(f'{folder_path}/labels.json') as f:
    y = json.load(f)
y = np.array([d['label'] for d in y])

In [ ]:
'''
n_samples = 2000
indices = np.random.choice([i for i in range(len(X_raw))], size=n_samples, replace=False)

X_raw = X_raw[indices]
y = y[indices]
'''
X_raw = X_raw[1000:2000]
y = y[1000:2000]

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()

X_raw = digits.images
X = digits.data
y = digits.target

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_raw = np.vstack((x_train, x_test))
X = X_raw.reshape(X_raw.shape[0], -1)
y = np.concatenate((y_train, y_test)).reshape(-1)

In [ ]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
X_raw = np.vstack((x_train, x_test))
X = X_raw.reshape(X_raw.shape[0], -1)
y = np.concatenate((y_train, y_test))

## Raw Data Points

- data source: flattened image
    - variation: the color space of the image (RGB, GRAY, HSV, LUV)
- postprocess: no postprocess

In [ ]:
from feature_extraction import raw_flatten

X, feature_names = raw_flatten(X_raw)

In [ ]:
from sklearn import decomposition

print('feature representation shape: ', X.shape)

reducer = decomposition.TruncatedSVD(n_components=2)
X_proj = reducer.fit_transform(X) if X.shape[1] > 2 else X

chart = alt.Chart(pd.DataFrame({
    'dim-1': X_proj[:, 0],
    'dim-2': X_proj[:, 1],
    'label': y,
})).mark_point().encode(
    x='dim-1:Q',
    y='dim-2:Q',
    color='label:N',
)
chart

## Dimension Reduction

- data source: flattened image
    - variation: the color space of the image (RGB, GRAY, HSV, LUV)
- postprocess: dimension reduction
    - variation
        - the dimension reduction method (PCA, MDS, t-SNE, isomap)
        - the number of dimensions

In [ ]:
from feature_extraction import resize_SVD

X, feature_names = resize_SVD(X_raw)

In [ ]:
from sklearn import decomposition

print('feature representation shape: ', X.shape)

reducer = decomposition.TruncatedSVD(n_components=2)
X_proj = reducer.fit_transform(X) if X.shape[1] > 2 else X

chart = alt.Chart(pd.DataFrame({
    'dim-1': X_proj[:, 0],
    'dim-2': X_proj[:, 1],
    'label': y,
})).mark_point().encode(
    x='dim-1:Q',
    y='dim-2:Q',
    color='label:N',
)
chart

## Feature Extraction with Pretrained CNN

- data source: image
- postprocess: feature extraction with CNN
    - variation
        - the pretrained model
        - the output layer

In [ ]:
def extract_cnn_output(imgs: np.ndarray) -> Tuple[np.ndarray, List[str]]:
    """
    Extract features for each image by convolution layer output of pretrained CNN.

    The images are converted to grey image and resized to 224 x 224
    to match the size of the original input to the CNN.
    
    The dimension reduction is conducted on the CNN output.

    Args
    ----
    imgs : np.ndarray
        The images to extract features.

    Returns
    -------
    X : np.ndarray
        The extracted feature values.
    feature_names : List[str]
        The names of features.
    """

    h, w = 224, 224
    imgs_resized = []
    for img in imgs:
        if len(img.shape) == 2:
            img_color = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
        else:
            img_color = img
        img_resized = cv.resize(img_color, (h, w), interpolation=cv.INTER_AREA)
        imgs_resized.append(img_resized)
    imgs_resized = np.array(imgs_resized)
    X_raw_processed = preprocess_input(imgs_resized.astype(float))
    
    model = VGG16(weights='imagenet', include_top=False)
    X_vgg16 = model.predict(X_raw_processed)
    n_samples = X_vgg16.shape[0]
    X_vgg16 = X_vgg16.reshape(n_samples, -1)

    dims = 50
    reducer = decomposition.TruncatedSVD(n_components=dims)
    X_proj = reducer.fit_transform(X)

    feature_names = [f'cnn-dim-{i}' for i in range(dims)]
    return X_proj, feature_names


In [ ]:
import cv2 as cv
from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16

model = VGG16(weights='imagenet', include_top=False)
target_size = (224, 224)

X_raw_processed = X_raw[: 10]

# turn gray image into color image
if len(X_raw.shape) == 3:
    X_raw_processed = np.array(list(map(lambda img: cv.cvtColor(img, cv.COLOR_GRAY2BGR),
                                    X_raw_processed)))
X_raw_processed = np.array(list(map(lambda img: cv.resize(img, target_size, interpolation=cv.INTER_AREA),
                                    X_raw_processed)))
X_raw_processed = preprocess_input(X_raw_processed.astype(float))

X_vgg16 = model.predict(X_raw_processed)
n_samples = X_vgg16.shape[0]
X_vgg16 = X_vgg16.reshape(n_samples, -1)

In [ ]:
print(f'shape of extracted feature representation: {X_vgg16.shape}')

reducer = decomposition.TruncatedSVD(n_components=2)
X_proj = reducer.fit_transform(X_vgg16)

n_samples = 1000
indices = np.random.choice([i for i in range(len(X))], size=n_samples, replace=False)

chart = alt.Chart(pd.DataFrame({
    'dim-1': X_proj[:, 0],
    'dim-2': X_proj[:, 1],
    'label': y[:, 1000],
})).mark_point().encode(
    x='dim-1:Q',
    y='dim-2:Q',
    color='label:N',
)
chart

## Color Features

- data source: flattened image
- postprocess: statistical measurements
    - variation
        - histogram
        - median
        - moments
            - mean
            - std
            - skewness
        - correlogram
        - coherence vector
    - variation
        - the color space
    - variation
        - value quantization
- note: the features can be computed in different color spaces (e.g., RGB, HSV, LUV)
- remark: Hu moments often output nan which is not desirable

In [ ]:
from feature_extraction import color_descriptors

X, feature_names = color_descriptors(X_raw)

In [ ]:
from scipy.stats import pearsonr
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder().fit(y)
y_encoded = encoder.transform(y)
rvalues = [pearsonr(X[:, i], y_encoded)[0] for i in range(len(feature_names))]
for i in range(len(feature_names)):
    print(f'{feature_names[i]} corr-y: {rvalues[i]}')

In [ ]:
from sklearn import decomposition

print('feature representation shape: ', X.shape)

reducer = decomposition.TruncatedSVD(n_components=2)
X_proj = reducer.fit_transform(X) if X.shape[1] > 2 else X

chart = alt.Chart(pd.DataFrame({
    'dim-1': X_proj[:, 0][600:],
    'dim-2': X_proj[:, 1][600:],
    'label': y[600:],
})).mark_point().encode(
    x='dim-1:Q',
    y='dim-2:Q',
    color='label:N',
)
chart

In [ ]:
from sklearn import decomposition, manifold

hist_feature_indices = [i for i in range(len(feature_names)) if feature_names[i].find('hist') != -1]

print('feature representation shape: ', X[:, hist_feature_indices].shape)

#reducer = decomposition.TruncatedSVD(n_components=2)
reducer = manifold.TSNE(n_components=2, random_state=0)
X_proj = reducer.fit_transform(X[:, hist_feature_indices])

chart = alt.Chart(pd.DataFrame({
    'dim-1': X_proj[:, 0][600:],
    'dim-2': X_proj[:, 1][600:],
    'label': y[600:],
})).mark_point().encode(
    x='dim-1:Q',
    y='dim-2:Q',
    color='label:N',
).properties(
    title='color histogram features'
)
chart

In [ ]:
from sklearn import decomposition, manifold

nonhist_feature_indices = [i for i in range(len(feature_names)) if feature_names[i].find('hist') == -1]

print('feature representation shape: ', X[:, nonhist_feature_indices].shape)

#reducer = decomposition.TruncatedSVD(n_components=2)
reducer = manifold.TSNE(n_components=2, random_state=0)
X_proj = reducer.fit_transform(X[:, nonhist_feature_indices])

chart = alt.Chart(pd.DataFrame({
    'dim-1': X_proj[:, 0][600:],
    'dim-2': X_proj[:, 1][600:],
    'label': y[600:],
})).mark_point().encode(
    x='dim-1:Q',
    y='dim-2:Q',
    color='label:N',
).properties(
    title='color statistics features'
)
chart

In [ ]:
# quantized histogram

from sklearn.cluster import KMeans
from sklearn.utils import shuffle

n_samples = 10000
n_colors = 50

colors = X_raw.reshape(-1, 3)
#colors_sample = shuffle(colors, random_state=0)[:n_samples]
#indices = np.random.choice([i for i in range(len(colors))], size=n_samples, replace=False)
# note: generate random sampling indices with replacement for speed consideration
indices = [np.random.randint(0, len(colors)-1) for i in range(n_samples)]
colors_sample = colors[indices]
kmeans = KMeans(n_clusters=n_colors, random_state=0).fit(colors_sample)
cluster_centers = kmeans.cluster_centers_

In [ ]:
# correlogram

# https://medium.com/@shashwat17103/the-dummys-guide-to-colour-correlogram-from-scratch-in-python-1b20a55eb00c

# coherence vector

def ccv(src, tau=0, n=64):
    """
    Proccess of Computing CCV(color coherence vector)

    1. Blur
    2. Quantizing color
    3. Thresholding
    4. Labeling
    5. Counting
    """

    img = src.copy()
    row, col, channels = img.shape
    if not col == 300:
        aspect = 300.0//col
        img = cv2.resize(img, None, fx=aspect, fy=aspect, interpolation = cv2.INTER_CUBIC)
    row, col, channels = img.shape
    # blur
    img = cv2.GaussianBlur(img, (3,3),0)
    # quantize color
    img = QuantizeColor(img, n)
    bgr = cv2.split(img)
    #bgr = cv2.split(cv2.cvtColor(img, cv2.COLOR_BGR2HSV))
    if tau == 0:
        tau = row*col * 0.1
    alpha = np.zeros(n)
    beta = np.zeros(n)
    # labeling
    for i,ch in enumerate(bgr):
        ret,th = cv2.threshold(ch,127,255,0)
        ret, labeled, stat, centroids = cv2.connectedComponentsWithStats(th, None, cv2.CC_STAT_AREA, None, connectivity=8)
        # generate ccv
        areas = [[v[4],label_idx] for label_idx,v in enumerate(stat)]
        coord = [[v[0],v[1]] for label_idx,v in enumerate(stat)]
        # Counting
        for a,c in zip(areas, coord):
        area_size = a[0]
        x,y = c[0], c[1]
        if (x < ch.shape[1]) and (y < ch.shape[0]):
            bin_idx = int(ch[y,x]//(256//n))
            if area_size >= tau:
            alpha[bin_idx] = alpha[bin_idx] + area_size
            else:
            beta[bin_idx] = beta[bin_idx] + area_size
    return alpha, beta

## Edge Strength Features

- data source: flattened edge image
- postprocess: statistical measurements
    - variation
        - histogram
        - moments
        - correlogram
        - coherence vector
    - variation
        - the edge detection method
    - variation
        - value quantization

In [ ]:
from feature_extraction import edge_descriptors

X, feature_names = edge_descriptors(X_raw)

In [ ]:
from scipy.stats import pearsonr
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder().fit(y)
y_encoded = encoder.transform(y)
rvalues = [pearsonr(X[:, i], y_encoded)[0] for i in range(len(feature_names))]
for i in range(len(feature_names)):
    print(f'{feature_names[i]} corr-y: {rvalues[i]}')

In [ ]:
from sklearn import decomposition

print('feature representation shape: ', X.shape)

reducer = decomposition.TruncatedSVD(n_components=2)
X_proj = reducer.fit_transform(X) if X.shape[1] > 2 else X

chart = alt.Chart(pd.DataFrame({
    'dim-1': X_proj[:, 0],
    'dim-2': X_proj[:, 1],
    'label': y,
})).mark_point().encode(
    x='dim-1:Q',
    y='dim-2:Q',
    color='label:N',
).properties(
    title='edge features'
)
chart

In [ ]:
from sklearn import decomposition, manifold

hist_feature_indices = [i for i in range(len(feature_names)) if feature_names[i].find('hist') != -1]

print('feature representation shape: ', X[:, hist_feature_indices].shape)

reducer = decomposition.TruncatedSVD(n_components=2)
#reducer = manifold.TSNE(n_components=2, random_state=0)
X_proj = reducer.fit_transform(X[:, hist_feature_indices])

chart = alt.Chart(pd.DataFrame({
    'dim-1': X_proj[:, 0][600:],
    'dim-2': X_proj[:, 1][600:],
    'label': y[600:],
})).mark_point().encode(
    x='dim-1:Q',
    y='dim-2:Q',
    color='label:N',
).properties(
    title='edge histogram features'
)
chart

In [ ]:
from sklearn import decomposition, manifold

nonhist_feature_indices = [i for i in range(len(feature_names)) if feature_names[i].find('hist') == -1]

print('feature representation shape: ', X[:, nonhist_feature_indices].shape)

reducer = decomposition.TruncatedSVD(n_components=2)
#reducer = manifold.TSNE(n_components=2, random_state=0)
X_proj = reducer.fit_transform(X[:, nonhist_feature_indices])

chart = alt.Chart(pd.DataFrame({
    'dim-1': X_proj[:, 0][600:],
    'dim-2': X_proj[:, 1][600:],
    'label': y[600:],
})).mark_point().encode(
    x='dim-1:Q',
    y='dim-2:Q',
    color='label:N',
).properties(
    title='edge statistics features'
)
chart

## Edge Direction Features

- data source: flattened edge direction image
- postprocess: statistical measurements
    - variation
        - histogram
        - moments
        - correlogram
        - coherence vector
    - variation
        - the edge detection method
    - variation
        - value quantization

In [ ]:
from feature_extraction import edge_direction_descriptors

X, feature_names = edge_direction_descriptors(X_raw)

In [ ]:
from scipy.stats import pearsonr
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder().fit(y)
y_encoded = encoder.transform(y)
rvalues = [pearsonr(X[:, i], y_encoded)[0] for i in range(len(feature_names))]
for i in range(len(feature_names)):
    print(f'{feature_names[i]} corr-y: {rvalues[i]}')

In [ ]:
from sklearn import decomposition

print('feature representation shape: ', X.shape)

reducer = decomposition.TruncatedSVD(n_components=2)
X_proj = reducer.fit_transform(X) if X.shape[1] > 2 else X

chart = alt.Chart(pd.DataFrame({
    'dim-1': X_proj[:, 0],
    'dim-2': X_proj[:, 1],
    'label': y,
})).mark_point().encode(
    x='dim-1:Q',
    y='dim-2:Q',
    color='label:N',
).properties(
    title='edge direction features'
)
chart

In [ ]:
import math
from skimage.measure import moments

m = moments(img_gray, order=2)

# center
x = m[1, 0] / m[0, 0]
y = m[0, 1] / m[0, 0]

# central moments
a = m[2, 0] / m[0, 0] - x ** 2
b = 2 * (m[1, 1] / m[0, 0] - x * y)
c = m[0, 2] / m[0, 0] - y ** 2

# image orientation
if a == c:
    theta = math.pi / 2
else:
    theta = 1 / 2 * math.atan(b / (a - c)) + (a < c) * math.pi / 2
angle = theta / math.pi * 180

## Texture Features

- local binary patterns
- grey-level co-occurrence matrix
- frequency domain descriptors
- entropy
- local entropy

In [ ]:
from feature_extraction import texture_descriptors

X, feature_names = texture_descriptors(X_raw)

In [ ]:
from scipy.stats import pearsonr
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder().fit(y)
y_encoded = encoder.transform(y)
rvalues = [pearsonr(X[:, i], y_encoded)[0] for i in range(len(feature_names))]
for i in range(len(feature_names)):
    print(f'{feature_names[i]} corr-y: {rvalues[i]}')

In [ ]:
from sklearn import decomposition, manifold

print('feature representation shape: ', X.shape)

reducer = manifold.TSNE(n_components=2, random_state=0)
#reducer = decomposition.TruncatedSVD(n_components=2)
X_proj = reducer.fit_transform(X) if X.shape[1] > 2 else X

chart = alt.Chart(pd.DataFrame({
    'dim-1': X_proj[:, 0],
    'dim-2': X_proj[:, 1],
    'label': y,
})).mark_point().encode(
    x='dim-1:Q',
    y='dim-2:Q',
    color='label:N',
).properties(
    title='texture features'
)
chart

In [ ]:
import numpy as np

f = np.fft.fft2(img_gray)
fshift = np.fft.fftshift(f)
magnitude_spectrum = 20 * np.log(np.abs(fshift))

plt.subplot(121),plt.imshow(img_gray, cmap = 'gray')
plt.title('Input Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(magnitude_spectrum, cmap = 'gray')
plt.title('Magnitude Spectrum'), plt.xticks([]), plt.yticks([])
plt.show()

## Domain Specific Features

- TBA